# File Name: simple_rag_building_llama_index.ipynb
### Location: Chapter 6
### Purpose: 
#####       1. Create a Llama index
#####       2. Ingest and build the index
#####       3. Generating responses with RAG
##### Dependency: Not Applicable
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "langchain>=0.2.16" \
    "langchain_community>=0.2.17" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84" \
    "PyPDF2" \
    "pypdf" \
    "llama-index" \
    "llama-index-llms-bedrock" \
    "llama-index-embeddings-bedrock" \
    "llama-index-embeddings-huggingface" \
    "llama-index-llms-langchain" \
    "llama-index-embeddings-langchain" \
    "langchain-aws>=0.1.7" 

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import boto3
import botocore
import warnings
import time
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
from langchain_aws import ChatBedrock

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

# Find out data directory

#### 1. Retrieves the current working directory and prints it.
##### 2. Builds a path that navigates up one directory and appends 'data/rag_use_cases' to the path, then prints this resulting path.

In [ ]:
%%time
try:
    # Get the current working directory
    current_directory = os.getcwd()
    
    # Print the current working directory
    print(f"Current working directory: {current_directory}")
    
    # Construct the path to 'data/rag_use_cases' inside the current directory
    data_directory = os.path.join(current_directory, 'data', 'rag_use_cases')
    
    # Print the resulting path
    print(f"Data directory path: {data_directory}")
    
except FileNotFoundError as e:
    # Handle the case where the directory path does not exist
    print(f"Error: The specified path does not exist - {e}")
    
except Exception as e:
    # General exception handler for any other errors
    print(f"An unexpected error occurred: {e}")

# Disclaimer
##### Make Sure that data_directory is pointing to the right path and data files are present. Otherwise, you need to change the above code

# Define prompt, Amazon Bedrock Foundation model, and Amazon Bedrock embed model

In [ ]:
# Define prompt
prompt = "What is Amazon doing and cashflow?"

# List of Bedrock models with names and model codes
bedrock_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# List of Bedrock embed models with names and model codes
bedrock_embed_model_id = "amazon.titan-embed-text-v1"

## Define important environment variable

In [ ]:
%%time
# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    # This session is responsible for managing credentials and region configuration
    boto3_session = boto3.session.Session()

    # Retrieve the current AWS region from the session (e.g., 'us-east-1', 'us-west-2')
    aws_region_name = boto3_session.region_name
    
    # Initialize Bedrock and Bedrock Runtime clients using Boto3
    # These clients will allow interactions with Bedrock-related AWS services
    boto3_bedrock_client = boto3.client('bedrock', region_name=aws_region_name)
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=aws_region_name)

    # Store all relevant variables in a dictionary for easier access and management
    variables_store = {
        "aws_region_name": aws_region_name,                          # AWS region name
        "boto3_bedrock_client": boto3_bedrock_client,                # Bedrock client instance
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client,  # Bedrock Runtime client instance
        "boto3_session": boto3_session                               # Current Boto3 session object
    }

    # Print all stored variables for debugging and verification
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

# Handle any exceptions that occur during the execution
except Exception as e:
    # Print the error message if an unexpected error occurs
    print(f"An unexpected error occurred: {e}")

# Important package to consider for Llama Index 

In [ ]:
# Import necessary modules from LlamaIndex for VectorStore management and AWS Bedrock interaction
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

# Section 1

#  Embedding Model Initialization and LLaMA Settings Configuration

### 1. initialize_bedrock_embeddings Function:
##### Initializes the Bedrock Embeddings model using the provided Bedrock client and model ID.
##### Includes error handling to manage failures during initialization.

### 2. configure_llama_settings Function:
##### Configures global settings for LLaMA by assigning the LLM, embeddings model, and chunk size.
##### Error handling ensures that any issues during configuration are logged.

In [ ]:
%%time
# Function to initialize the Bedrock embeddings model
def initialize_bedrock_embeddings(bedrock_client, model_id):
    """
    Initializes the Bedrock Embeddings model using the provided Bedrock client and model ID.
    
    Args:
        bedrock_client: The initialized Bedrock client.
        model_id: The model ID for the embeddings model.
    
    Returns:
        embeddings_model: The initialized Bedrock Embeddings model or None if initialization fails.
    """
    try:
        # Attempt to initialize the embeddings model
        embeddings_model = BedrockEmbeddings(client=bedrock_client, model_id=model_id)
        print("Successfully initialized Bedrock Embeddings model.")
        return embeddings_model
    except Exception as e:
        # Handle errors in model initialization
        print(f"Error initializing Bedrock Embeddings model: {e}")
        return None

# Function to set up core settings for LLM and embeddings model
def configure_llama_settings(llm, embed_model, chunk_size=512):
    """
    Configures settings for LLM and the embeddings model, including chunk size.
    
    Args:
        llm: The initialized language model.
        embed_model: The initialized embeddings model.
        chunk_size: The chunk size for processing documents (default is 512).
    """
    try:
        # Configure the LLaMA settings globally
        Settings.llm = llm
        Settings.embed_model = embed_model
        Settings.chunk_size = chunk_size
        print(f"Settings configured: LLM={llm}, Embeddings Model={embed_model}, Chunk Size={chunk_size}")
    except Exception as e:
        # Handle errors in configuration
        print(f"Error configuring LLaMA settings: {e}")

# Example usage with modular try-catch blocks for setup
try:
    # Initialize Bedrock embeddings model
    embed_model = initialize_bedrock_embeddings(boto3_bedrock_runtime_client, bedrock_embed_model_id)
    if not embed_model:
        raise Exception("Bedrock embeddings model initialization failed.")

    # Initialize the LLM (replace with your LLM instance as required)
    llm = ChatBedrock(client=boto3_bedrock_runtime_client, model_id=bedrock_model_id)

    # Configure settings for LLaMA with the LLM and embeddings model
    configure_llama_settings(llm, embed_model, chunk_size=512)

except Exception as e:
    # Handle unexpected errors during the setup process
    print(f"An unexpected error occurred during setup: {e}")


# Loading Documents, Creating a Vector Store Index with llama index, Executing a Query with RAG Strategy

### 1. load_documents_from_directory:
##### Loads documents from a specified directory using the SimpleDirectoryReader.
##### It returns the documents if successful or None if an error occurs, with error handling included.

### 2. create_vector_store_index:
##### Creates a VectorStoreIndex from the loaded documents.
##### It returns the created index or None if there is an error.

### 3. execute_query:
##### Executes a query using the QueryEngine created from the index and returns the response.
##### Includes error handling for issues during query execution.

### 4. load_data_retrieve:
##### Main function that orchestrates the document loading, index creation, and query execution process.
##### If any step fails (loading documents, creating the index, or executing the query), it exits with appropriate messages.
##### If the process succeeds, it prints the final query response.

In [ ]:
%%time
# Function to load documents from a directory
def load_documents_from_directory(directory_path):
    """
    Loads documents from the specified directory using SimpleDirectoryReader.
    
    Args:
        directory_path (str): Path to the directory containing documents.
    
    Returns:
        list: List of loaded documents or None if an error occurs.
    """
    try:
        # Load documents from the specified directory
        documents = SimpleDirectoryReader(directory_path).load_data()
        print(f"Successfully loaded {len(documents)} documents from {directory_path}.")
        return documents
    except Exception as e:
        # Handle errors during document loading
        print(f"Error loading documents from {directory_path}: {e}")
        return None

# Function to create a VectorStore index from the loaded documents
def create_vector_store_index(documents):
    """
    Creates a VectorStoreIndex from the provided documents.
    
    Args:
        documents (list): List of documents to be indexed.
    
    Returns:
        VectorStoreIndex: The created index or None if an error occurs.
    """
    try:
        # Create a VectorStoreIndex from the provided documents
        index = VectorStoreIndex.from_documents(documents)
        print("VectorStore index created successfully.")
        return index
    except Exception as e:
        # Handle errors during index creation
        print(f"Error creating VectorStore index: {e}")
        return None

# Section 2

## Generating responses with RAG

In [ ]:
%%time
# Function to execute a query using the query engine
def execute_query(query_engine, query):
    """
    Executes a query on the query engine and returns the response.
    
    Args:
        query_engine (QueryEngine): The query engine to use for querying.
        query (str): The query string to execute.
    
    Returns:
        str: The query response or None if an error occurs.
    """
    try:
        # Query the index and return the response
        response = query_engine.query(query)
        print(f"Query executed successfully. Response: {response}")
        return response
    except Exception as e:
        # Handle errors during query execution
        print(f"Error executing query '{query}': {e}")
        return None

# Main function to run the process
def load_data_retrieve():
    """
    Main function to load documents, create the index, and execute a query.
    Handles the entire data retrieval process.
    """
    try:
        # Load documents
        documents = load_documents_from_directory(data_directory)

        if not documents:
            print("No documents loaded, exiting.")
            return

        # Create the index from loaded documents
        index = create_vector_store_index(documents)

        if not index:
            print("Index creation failed, exiting.")
            return

        # Convert index to a query engine
        query_engine = index.as_query_engine(similarity_top_k=3)

        # Execute query
        response = execute_query(query_engine, prompt)

        # Print the final response
        if response:
            print("Final response:", response)
    except Exception as e:
        # Handle any unexpected errors in the main process
        print(f"An unexpected error occurred in the main process: {e}")

# Execute the main section
load_data_retrieve()

# You shouod try to generate responses without RAG and compare the response with RAG result. 

# Generating responses from Amazon Bedrock foundation model without RAG

In [ ]:
%%time
# Initialize the language model
llm = ChatBedrock(
    model_id=bedrock_model_id,
    client=boto3_bedrock_runtime_client
)

# Define the system prompt
system_prompt = (
    "You are an assistant designed for answering questions. "
    "If you are unsure of the answer, please indicate that you "
    "do not know. Limit your response to a maximum of three sentences, "
    "ensuring that your answer is concise."
)

# Function to generate a response to a question
def generate_response(question):
    # Combine the system prompt and the user question
    input_text = f"{system_prompt}\n\n{question}"
    
    # Invoke the Bedrock model with the input text as a string
    response = llm.invoke(input_text)
    
    # Extract the content of the response
    return response

# Example usage
response = generate_response(prompt)

# Print the result
print(response)

# Must Read

######  Please have a look of the result of both the cases. Case 1 is generating more context aware information compare to Case 2.

###### Case1: Prompt enhancement with RAG and generating responses from Amazon Bedrock foundation model
###### Case2: Generating responses from Amazon Bedrock foundation model without RAG

# End of NoteBook 

## Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

## Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html